In [ ]:

import os
import shutil
import cv2
import numpy as np
import random
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from scipy.stats import entropy
import pandas as pd
import matplotlib.pyplot as plt

# Set seed for reproducibility
random.seed("This is a seed.")
np.random.seed(42)

# ============ PATH CONFIGURATION ============
BASE_PATH = "../../Photos_Subset/Original"

# Output paths for noised images
BW_OUTPUT = "../../Photos_Subset/Noised_Images/BW"
COLOUR_OUTPUT = "../../Photos_Subset/Noised_Images/Color"

# Output paths for noised CSVs
OUTPUT_SP = "../../Data_Results/Data/noised_images_data/salt_pepper"
OUTPUT_SPECKLE = "../../Data_Results/Data/noised_images_data/speckle"

# Bar graphs (histograms) output paths
BAR_GRAPHS_SP = "../../Data_Results/Data/bar_graphs/salt_pepper"
BAR_GRAPHS_SPECKLE = "../../Data_Results/Data/bar_graphs/speckle"

# BAR_GRAPHS_SP, BAR_GRAPHS_SPECKLE

# os.makedirs(BAR_GRAPHS_SP, exist_ok=True)
# os.makedirs(BAR_GRAPHS_SPECKLE, exist_ok=True)

# Clean output directories if they exist
# for output_dir in [BW_OUTPUT, COLOUR_OUTPUT, OUTPUT_SP, OUTPUT_SPECKLE, 
#                    ]:
#     if os.path.exists(output_dir):
#         print(f"Cleaning existing output directory: {output_dir}")
#         shutil.rmtree(output_dir)
#     os.makedirs(output_dir, exist_ok=True)

print(f"✓ Base (original) images: {BASE_PATH}")
print(f"✓ BW noised output: {BW_OUTPUT}")
print(f"✓ Colour noised output: {COLOUR_OUTPUT}")
print(f"✓ Salt & Pepper CSV output: {OUTPUT_SP}")
print(f"✓ Speckle CSV output: {OUTPUT_SPECKLE}")
print(f"✓ Salt & Pepper bar graphs: {BAR_GRAPHS_SP}")
print(f"✓ Speckle bar graphs: {BAR_GRAPHS_SPECKLE}\n")

# ---------------------------- Noise functions with parameters ----------------------------
def add_salt_pepper_noise(image, amount=0.2, salt_vs_pepper=0.5):
    """
    Add salt and pepper noise
    amount: proportion of image to be noised (0 to 1)
    salt_vs_pepper: proportion of salt vs pepper (0.5 = equal amounts)
    """
    noisy = np.copy(image)
    num_salt = np.ceil(amount * image.size * salt_vs_pepper)
    num_pepper = np.ceil(amount * image.size * (1 - salt_vs_pepper))

    # Add salt (white pixels)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape[:2]]
    noisy[coords[0], coords[1]] = 255

    # Add pepper (black pixels)
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape[:2]]
    noisy[coords[0], coords[1]] = 0

    return noisy

def add_speckle_noise(image, intensity=1.0):
    """
    Add speckle noise
    intensity: multiplier for noise strength (typical range: 0.1 to 5.0)
    """
    row, col, ch = image.shape
    gauss = np.random.randn(row, col, ch)
    noisy = image + image * gauss * intensity
    return np.clip(noisy, 0, 255).astype(np.uint8)

# ---------------------------- Comprehensive Metric Calculation ----------------------------
def calculate_entropy(image):
    """Calculate entropy of image"""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    hist, _ = np.histogram(gray.flatten(), bins=256, range=(0, 256))
    hist = hist / hist.sum()
    hist = hist[hist > 0]
    return entropy(hist, base=2)

def calculate_sharpness(image):
    """Calculate sharpness using Laplacian variance"""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    return laplacian.var()

def calculate_spatial_frequency(image):
    """Calculate spatial frequency"""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    gray = gray.astype(np.float64)
    RF = np.sqrt(np.mean(np.diff(gray, axis=1) ** 2))
    CF = np.sqrt(np.mean(np.diff(gray, axis=0) ** 2))
    SF = np.sqrt(RF ** 2 + CF ** 2)

    return SF

def calculate_dynamic_range(image):
    """Calculate dynamic range"""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    return np.max(gray) - np.min(gray)

def calculate_noise_variance(original, noisy):
    """Calculate variance of the noise"""
    if len(original.shape) == 3:
        original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
        noisy_gray = cv2.cvtColor(noisy, cv2.COLOR_BGR2GRAY)
    else:
        original_gray = original
        noisy_gray = noisy

    noise = noisy_gray.astype(np.float64) - original_gray.astype(np.float64)
    return np.var(noise)

def calculate_all_metrics(original, noisy):
    """Calculate all metrics between original and noisy images"""
    if len(original.shape) == 3:
        original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
        noisy_gray = cv2.cvtColor(noisy, cv2.COLOR_BGR2GRAY)
    else:
        original_gray = original
        noisy_gray = noisy

    psnr_value = psnr(original_gray, noisy_gray, data_range=255)
    ssim_value = ssim(original_gray, noisy_gray, data_range=255)
    mse_value = np.mean((original_gray.astype(np.float64) - noisy_gray.astype(np.float64)) ** 2)

    entropy_orig = calculate_entropy(original)
    entropy_noisy = calculate_entropy(noisy)
    entropy_diff = abs(entropy_noisy - entropy_orig)

    noise_var = calculate_noise_variance(original, noisy)
    sharpness_value = calculate_sharpness(noisy)
    spatial_freq = calculate_spatial_frequency(noisy)
    dynamic_range = calculate_dynamic_range(noisy)

    return {
        "PSNR": psnr_value,
        "SSIM": ssim_value,
        "MSE": mse_value,
        "Entropy_Diff": entropy_diff,
        "Noise_Variance": noise_var,
        "Sharpness": sharpness_value,
        "Spatial_Freq": spatial_freq,
        "Dynamic_Range": dynamic_range
    }

# ======================== HISTOGRAM FUNCTION ========================
def create_noisy_histogram(df, histogram_path, image_type):
    """Create histogram for noisy image metrics"""
    try:
        metrics = ["PSNR", "SSIM", "MSE", "Entropy_Diff", "Noise_Variance", 
                   "Sharpness", "Spatial_Freq", "Dynamic_Range"]
        
        fig, axes = plt.subplots(4, 2, figsize=(14, 16))
        axes = axes.flatten()
        
        for idx, metric in enumerate(metrics):
            ax = axes[idx]
            data = df[metric].values
            
            n, bins, patches = ax.hist(
                data, bins=20, color='steelblue', 
                edgecolor='black', alpha=0.7
            )
            
            # Color bars based on metric type
            if metric in ['PSNR', 'SSIM', 'Sharpness', 'Spatial_Freq', 'Dynamic_Range']:
                color = 'green'  # Higher is better
            else:
                color = 'orange'  # Lower is better
            
            for patch in patches:
                patch.set_facecolor(color)
            
            # Add mean line
            mean_val = data.mean()
            ax.axvline(mean_val, color='red', linestyle='--', linewidth=2, 
                      label=f'Mean: {mean_val:.2f}')
            
            # Add median line
            median_val = np.median(data)
            ax.axvline(median_val, color='blue', linestyle='--', linewidth=2,
                      label=f'Median: {median_val:.2f}')
            
            ax.set_xlabel(f'{metric} Value', fontsize=10, fontweight='bold')
            ax.set_ylabel('Frequency', fontsize=10, fontweight='bold')
            ax.set_title(f'{metric} Distribution', fontsize=11, fontweight='bold')
            ax.legend(fontsize=9)
            ax.grid(axis='y', alpha=0.3)
        
        plt.suptitle(f'Noisy Image Metrics - {image_type}', 
                    fontsize=14, fontweight='bold')
        plt.tight_layout()
        
        plt.savefig(histogram_path, dpi=300, bbox_inches='tight')
        plt.close()
        
    except Exception as e:
        print(f"      Error creating histogram: {e}")

# ---------------------------- Parameter configurations ----------------------------
SALT_PEPPER_PARAMS = [
    {"amount": 0.01, "salt_vs_pepper": 0.5, "name": "amount_0.01"},
    {"amount": 0.05, "salt_vs_pepper": 0.5, "name": "amount_0.05"},
    {"amount": 0.1, "salt_vs_pepper": 0.5, "name": "amount_0.10"},
    {"amount": 0.15, "salt_vs_pepper": 0.5, "name": "amount_0.15"},
    {"amount": 0.2, "salt_vs_pepper": 0.5, "name": "amount_0.20"},
]

SPECKLE_PARAMS = [
    {"intensity": 0.5, "name": "intensity_0.5"},
    {"intensity": 1.0, "name": "intensity_1.0"},
    {"intensity": 1.5, "name": "intensity_1.5"},
    {"intensity": 2.0, "name": "intensity_2.0"},
    {"intensity": 2.5, "name": "intensity_2.5"},
    {"intensity": 3.0, "name": "intensity_3.0"},
]

# ---------------------------- Verify base_path exists ----------------------------
if not os.path.exists(BASE_PATH):
    raise FileNotFoundError(f"Base path does not exist: {BASE_PATH}")

print(f"Base path verified: {BASE_PATH}")
print(f'Random seed set to: "This is a seed."\n')

# Get all image files
all_files = [f for f in os.listdir(BASE_PATH)
             if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff"))
             and os.path.isfile(os.path.join(BASE_PATH, f))]

# Separate color and BW files
color_files = []
bw_files = []

for filename in all_files:
    if filename[-6:-4] == 'bw' or filename.lower().endswith('_bw.jpg') or filename.lower().endswith('_bw.png'):
        bw_files.append(filename)
    else:
        color_files.append(filename)

print(f"Found {len(color_files)} color images")
print(f"Found {len(bw_files)} BW images")
print(f"Total: {len(all_files)} images\n")

# ---------------------------- Process Salt & Pepper Noise ----------------------------
print("="*80)
print("PROCESSING SALT & PEPPER NOISE")
print("="*80)

for param_config in SALT_PEPPER_PARAMS:
    param_name = param_config["name"]
    amount = param_config["amount"]
    salt_vs_pepper = param_config["salt_vs_pepper"]

    print(f"\nProcessing: {param_name} (amount={amount})")

    # Create output directories
    bw_output_dir = os.path.join(BW_OUTPUT, param_name)
    color_output_dir = os.path.join(COLOUR_OUTPUT, param_name)
    os.makedirs(bw_output_dir, exist_ok=True)
    os.makedirs(color_output_dir, exist_ok=True)

    # Create histogram directory with _noisy suffix
    histogram_dir = os.path.join(BAR_GRAPHS_SP, f"{param_name}_noisy")
    os.makedirs(histogram_dir, exist_ok=True)

    # Store metrics
    metrics_color = []
    metrics_bw = []

    # Process COLOR images
    print(f"  Processing color images...")
    for filename in tqdm(color_files, desc=f"    Color {param_name}"):
        filepath = os.path.join(BASE_PATH, filename)
        img = cv2.imread(filepath)

        if img is None:
            continue

        # Add noise
        noisy_img = add_salt_pepper_noise(img, amount=amount, salt_vs_pepper=salt_vs_pepper)

        # Save
        output_filename = f"Noisy_{filename}"
        output_path = os.path.join(color_output_dir, output_filename)
        cv2.imwrite(output_path, noisy_img)

        # Calculate metrics
        metrics = calculate_all_metrics(img, noisy_img)
        metrics["Name"] = output_filename
        metrics_color.append(metrics)

    # Process BW images
    print(f"  Processing BW images...")
    for filename in tqdm(bw_files, desc=f"    BW {param_name}"):
        filepath = os.path.join(BASE_PATH, filename)
        img = cv2.imread(filepath)

        if img is None:
            continue

        # Add noise
        noisy_img = add_salt_pepper_noise(img, amount=amount, salt_vs_pepper=salt_vs_pepper)

        # Save
        output_filename = f"Noisy_{filename}"
        output_path = os.path.join(bw_output_dir, output_filename)
        cv2.imwrite(output_path, noisy_img)

        # Calculate metrics
        metrics = calculate_all_metrics(img, noisy_img)
        metrics["Name"] = output_filename
        metrics_bw.append(metrics)

    # Save CSV and create histograms - COLOR
    if metrics_color:
        df_color = pd.DataFrame(metrics_color)
        column_order = ["Name", "PSNR", "SSIM", "MSE", "Entropy_Diff",
                        "Noise_Variance", "Sharpness", "Spatial_Freq", "Dynamic_Range"]
        df_color = df_color[column_order]
        
        # Save CSV
        csv_dir = os.path.join(OUTPUT_SP, param_name)
        os.makedirs(csv_dir, exist_ok=True)
        csv_path_color = os.path.join(csv_dir, f"Noisy_{param_name}_color.csv")
        df_color.to_csv(csv_path_color, index=False)
        print(f"  Saved CSV: {csv_path_color}")
        
        # Create histogram
        histogram_path = os.path.join(histogram_dir, "color_histogram.png")
        create_noisy_histogram(df_color, histogram_path, f"{param_name} - Color")
        print(f"  Saved histogram: {histogram_path}")

    # Save CSV and create histograms - BW
    if metrics_bw:
        df_bw = pd.DataFrame(metrics_bw)
        df_bw = df_bw[column_order]
        
        # Save CSV
        csv_dir = os.path.join(OUTPUT_SP, param_name)
        os.makedirs(csv_dir, exist_ok=True)
        csv_path_bw = os.path.join(csv_dir, f"Noisy_{param_name}_bw.csv")
        df_bw.to_csv(csv_path_bw, index=False)
        print(f"  Saved CSV: {csv_path_bw}")
        
        # Create histogram
        histogram_path = os.path.join(histogram_dir, "bw_histogram.png")
        create_noisy_histogram(df_bw, histogram_path, f"{param_name} - BW")
        print(f"  Saved histogram: {histogram_path}")

# ---------------------------- Process Speckle Noise ----------------------------


for param_config in SPECKLE_PARAMS:
    param_name = param_config["name"]
    intensity = param_config["intensity"]

    print(f"\nProcessing: {param_name} (intensity={intensity})")

    # Create output directories
    bw_output_dir = os.path.join(BW_OUTPUT, param_name)
    color_output_dir = os.path.join(COLOUR_OUTPUT, param_name)
    os.makedirs(bw_output_dir, exist_ok=True)
    os.makedirs(color_output_dir, exist_ok=True)

    # Create histogram directory with _noisy suffix
    histogram_dir = os.path.join(BAR_GRAPHS_SPECKLE, f"{param_name}_noisy")
    os.makedirs(histogram_dir, exist_ok=True)

    # Store metrics
    metrics_color = []
    metrics_bw = []

    # Process COLOR images
    print(f"  Processing color images...")
    for filename in tqdm(color_files, desc=f"    Color {param_name}"):
        filepath = os.path.join(BASE_PATH, filename)
        img = cv2.imread(filepath)

        if img is None:
            continue

        # Add noise
        noisy_img = add_speckle_noise(img, intensity=intensity)

        # Save
        output_filename = f"Noisy_{filename}"
        output_path = os.path.join(color_output_dir, output_filename)
        cv2.imwrite(output_path, noisy_img)

        # Calculate metrics
        metrics = calculate_all_metrics(img, noisy_img)
        metrics["Name"] = output_filename
        metrics_color.append(metrics)

    # Process BW images
    print(f"  Processing BW images...")
    for filename in tqdm(bw_files, desc=f"    BW {param_name}"):
        filepath = os.path.join(BASE_PATH, filename)
        img = cv2.imread(filepath)

        if img is None:
            continue

        # Add noise
        noisy_img = add_speckle_noise(img, intensity=intensity)

        # Save
        output_filename = f"Noisy_{filename}"
        output_path = os.path.join(bw_output_dir, output_filename)
        cv2.imwrite(output_path, noisy_img)

        # Calculate metrics
        metrics = calculate_all_metrics(img, noisy_img)
        metrics["Name"] = output_filename
        metrics_bw.append(metrics)

    # Save CSV and create histograms - COLOR
    if metrics_color:
        df_color = pd.DataFrame(metrics_color)
        column_order = ["Name", "PSNR", "SSIM", "MSE", "Entropy_Diff",
                        "Noise_Variance", "Sharpness", "Spatial_Freq", "Dynamic_Range"]
        df_color = df_color[column_order]
        
        # Save CSV
        csv_dir = os.path.join(OUTPUT_SPECKLE, param_name)
        os.makedirs(csv_dir, exist_ok=True)
        csv_path_color = os.path.join(csv_dir, f"Noisy_{param_name}_color.csv")
        df_color.to_csv(csv_path_color, index=False)
        print(f"  Saved CSV: {csv_path_color}")
        
        # Create histogram
        histogram_path = os.path.join(histogram_dir, "color_histogram.png")
        create_noisy_histogram(df_color, histogram_path, f"{param_name} - Color")
        print(f"  Saved histogram: {histogram_path}")

    # Save CSV and create histograms - BW
    if metrics_bw:
        df_bw = pd.DataFrame(metrics_bw)
        df_bw = df_bw[column_order]
        
        # Save CSV
        csv_dir = os.path.join(OUTPUT_SPECKLE, param_name)
        os.makedirs(csv_dir, exist_ok=True)
        csv_path_bw = os.path.join(csv_dir, f"Noisy_{param_name}_bw.csv")
        df_bw.to_csv(csv_path_bw, index=False)
        print(f"  Saved CSV: {csv_path_bw}")
        
        # Create histogram
        histogram_path = os.path.join(histogram_dir, "bw_histogram.png")
        create_noisy_histogram(df_bw, histogram_path, f"{param_name} - BW")
        print(f"  Saved histogram: {histogram_path}")

# ---------------------------- Summary ----------------------------

print(f"Total parameter configurations tested:")
print(f"  Salt & Pepper: {len(SALT_PEPPER_PARAMS)} configurations")
print(f"  Speckle: {len(SPECKLE_PARAMS)} configurations")
print(f"\nImages processed:")
print(f"  Color: {len(color_files)}")
print(f"  BW: {len(bw_files)}")
print(f"\nOutputs saved to:")
print(f"  Noised images (BW): {BW_OUTPUT}")
print(f"  Noised images (Color): {COLOUR_OUTPUT}")
print(f"  CSVs (Salt & Pepper): {OUTPUT_SP}")
print(f"  CSVs (Speckle): {OUTPUT_SPECKLE}")
print(f"  Histograms (Salt & Pepper): {BAR_GRAPHS_SP}")
print(f"  Histograms (Speckle): {BAR_GRAPHS_SPECKLE}")
print("\n✓ Done!")


Cleaning existing output directory: ../../Photos_Subset/Noised_Images/BW
Cleaning existing output directory: ../../Photos_Subset/Noised_Images/Color
Cleaning existing output directory: ../../Data_Results/Data/noised_images_data/salt_pepper
Cleaning existing output directory: ../../Data_Results/Data/noised_images_data/speckle
✓ Base (original) images: ../../Photos_Subset/Original
✓ BW noised output: ../../Photos_Subset/Noised_Images/BW
✓ Colour noised output: ../../Photos_Subset/Noised_Images/Color
✓ Salt & Pepper CSV output: ../../Data_Results/Data/noised_images_data/salt_pepper
✓ Speckle CSV output: ../../Data_Results/Data/noised_images_data/speckle
✓ Salt & Pepper bar graphs: ../../Data_Results/Data/bar_graphs/salt_pepper
✓ Speckle bar graphs: ../../Data_Results/Data/bar_graphs/speckle

Base path verified: ../../Photos_Subset/Original
Random seed set to: "This is a seed."

Found 80 color images
Found 80 BW images
Total: 160 images

PROCESSING SALT & PEPPER NOISE

Processing: amount_0

    Color amount_0.01: 100%|██████████| 80/80 [00:01<00:00, 55.32it/s]


  Processing BW images...


    BW amount_0.01: 100%|██████████| 80/80 [00:01<00:00, 64.61it/s] 


  Saved CSV: ../../Data_Results/Data/noised_images_data/salt_pepper/amount_0.01/Noisy_amount_0.01_color.csv
  Saved histogram: ../../Data_Results/Data/bar_graphs/salt_pepper/amount_0.01_noisy/color_histogram.png
  Saved CSV: ../../Data_Results/Data/noised_images_data/salt_pepper/amount_0.01/Noisy_amount_0.01_bw.csv
  Saved histogram: ../../Data_Results/Data/bar_graphs/salt_pepper/amount_0.01_noisy/bw_histogram.png

Processing: amount_0.05 (amount=0.05)
  Processing color images...


...

  Saved CSV: ../../Data_Results/Data/noised_images_data/speckle/intensity_2.5/Noisy_intensity_2.5_color.csv
  Saved histogram: ../../Data_Results/Data/bar_graphs/speckle/intensity_2.5_noisy/color_histogram.png
  Saved CSV: ../../Data_Results/Data/noised_images_data/speckle/intensity_2.5/Noisy_intensity_2.5_bw.csv
  Saved histogram: ../../Data_Results/Data/bar_graphs/speckle/intensity_2.5_noisy/bw_histogram.png

Processing: intensity_3.0 (intensity=3.0)
  Processing color images...


    Color intensity_3.0: 100%|██████████| 80/80 [00:01<00:00, 43.95it/s]


  Processing BW images...


    BW intensity_3.0: 100%|██████████| 80/80 [00:01<00:00, 42.63it/s]


  Saved CSV: ../../Data_Results/Data/noised_images_data/speckle/intensity_3.0/Noisy_intensity_3.0_color.csv
  Saved histogram: ../../Data_Results/Data/bar_graphs/speckle/intensity_3.0_noisy/color_histogram.png
  Saved CSV: ../../Data_Results/Data/noised_images_data/speckle/intensity_3.0/Noisy_intensity_3.0_bw.csv
  Saved histogram: ../../Data_Results/Data/bar_graphs/speckle/intensity_3.0_noisy/bw_histogram.png
Total parameter configurations tested:
  Salt & Pepper: 5 configurations
  Speckle: 6 configurations

Images processed:
  Color: 80
  BW: 80

Outputs saved to:
  Noised images (BW): ../../Photos_Subset/Noised_Images/BW
  Noised images (Color): ../../Photos_Subset/Noised_Images/Color
  CSVs (Salt & Pepper): ../../Data_Results/Data/noised_images_data/salt_pepper
  CSVs (Speckle): ../../Data_Results/Data/noised_images_data/speckle
  Histograms (Salt & Pepper): ../../Data_Results/Data/bar_graphs/salt_pepper
  Histograms (Speckle): ../../Data_Results/Data/bar_graphs/speckle

✓ Done!
